In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib agg

In [ ]:
import os, sys, re
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), "lib/python"))

import numpy as np
import matplotlib.animation as ani

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader

In [ ]:
def latex_float(number):
    float_str = "{0:.3g}".format(number)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0:.2f}\times10^{{{1}}}".format(float(base), int(exponent))
    else:
        return float_str

In [ ]:
# service options
delimiter = '/'

## configuration options
data_path = '.'

if os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, verbose=False)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

time_range = [reader.meta.time[0], reader.meta.time[1]]
frame_step = 1
frame_size = [0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]

cmap='terrain'
image_interpolation = 'nearest'

autoselect = True
show_grid = False
aspect='equal'

guess_number_ticks=20

cmp = 'E/r'
specie = None # simply init with none. Replace with required specie for specied grids
probe_path = None

dry_run = False
view = False

if specie:
    probe_path = cmp + delimiter + specie
else:
    probe_path = cmp

video_file = "mesh_movie_{}_{}-{}.avi".format(re.sub(delimiter, '-', probe_path), time_range[0], time_range[1])

r_scale = reader.meta.geometry_size[0] / reader.meta.geometry_grid[0]
z_scale = reader.meta.geometry_size[1] / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = reader.meta.get_clim_estimation()
clim = [-clim_estimation, clim_estimation]

x_axis_label = r'$\mathit{z\enspace (m)}$'
y_axis_label = r'$\mathit{r\enspace (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'

plot_name = r"$\mathbf{{Mesh\enspace Color\enspace Map\enspace for\enspace {x}}}$".format(x=probe_path)

In [ ]:
# define plot builder
plot = PlotBuilder(frame_size[3] - frame_size[1], frame_size[2] - frame_size[0],
                   fig_color=reader.meta.figure_color, 
                   fig_width=7,
                   fig_height=1.6,
                   fig_dpi=300,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=8,
                   
                   x_ticklabel_start=frame_size[1] * z_scale, 
                   y_ticklabel_start=frame_size[0] * r_scale,
                   x_ticklabel_end=frame_size[3] * z_scale, 
                   y_ticklabel_end=frame_size[2] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect=aspect, image_interpolation=image_interpolation,
                   guess_number_ticks=guess_number_ticks)
o_left=0.100
o_right=0.895
o_top=0.860
o_bot=0.210

import matplotlib.pyplot as plt

plt.subplots_adjust(left=o_left,
                    bottom=o_bot,
                    right=o_right,
                    top=o_top,
                    wspace=0.010, 
                    hspace=0.010)


In [ ]:
# add subplots
subplot = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
subplot.yaxis.set_label_coords(-.09, .5)
subplot.xaxis.set_label_coords(.5, -.3)


# add timestamp to plot
# plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.80, y=.95)

# add initial image
initial_image = np.zeros([frame_size[2] - frame_size[0], frame_size[3] - frame_size[1]])
plot.add_image(plot_name, initial_image, cmap=cmap, clim=clim)
plot.add_colorbar(plot_name, ticks=clim, title=cbar_axis_label)

In [ ]:
schedule = 0

for p in reader.meta.probes:
    if p.component == cmp and (specie == None or p.specie == specie):
        schedule = p.schedule
        break
        
start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], schedule)
end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], schedule)

frame_src_size=[-1, -1, -1, -1]
    
# detect probe shape
for probe in reader.meta.probes:
    if ( probe.shape == 'rec'
        and probe.component == cmp
        and (specie == None or probe.specie == specie)
        and probe.size[0] == frame_size[0]
        and probe.size[1] == frame_size[1]
        and probe.size[2] == frame_size[2]
        and probe.size[3] == frame_size[3] ):
        frame_src_size = probe.size
        
    # try bigger frames, if autoselect enabled
    if ( ( frame_src_size[0] == -1 
          or frame_src_size[1] == -1 
          or frame_src_size[2] == -1 
          or frame_src_size[3] == -1 )
        and autoselect ):
        for probe in reader.meta.probes:
            if ( probe.shape == 'rec'
                and probe.component == cmp
                and (specie == None or probe.specie == specie)
                and probe.size[0] <= frame_size[0]
                and probe.size[1] <= frame_size[1]
                and probe.size[2] >= frame_size[2]
                and probe.size[3] >= frame_size[3] ):
                frame_src_size = probe.size

In [ ]:
FFMpegWriter = ani.writers['ffmpeg']

metadata = dict(title = plot_name)
writer = FFMpegWriter(fps = reader.meta.video_fps,
                      metadata = metadata,
                      codec = reader.meta.video_codec,
                      bitrate = reader.meta.video_bitrate)

if dry_run: video_file = '/dev/null'
fig = plot.get_figure()

with writer.saving(fig, video_file, 300):
    for i in range(start_frame, end_frame):
        if i % frame_step == 0:
            sys.stdout.write('Loading dataset ' + str(i) + '... ')
            sys.stdout.flush()
            data = reader.rec(probe_path, frame_src_size, i)[frame_size[0]:frame_size[2], frame_size[1]:frame_size[3]]

# dirty hack. Something wrong with particle placement
            if cmp == 'density' or cmp == 'temperature':
                data[128] = data[127]
                data[256] = data[255]
                # data[384] = data[383]
                data[:,128] = data[:,127]
                data[:,256] = data[:,255]
                data[:,384] = data[:,383]
                data[:,512] = data[:,511]
                data[:,640] = data[:,639]
                data[:,768] = data[:,767]
                data[:,896] = data[:,895]
                data[:,1024] = data[:,1023]
                data[:,1152] = data[:,1151]
                data[:,1280] = data[:,1279]
                data[:,1408] = data[:,1407]
                data[:,1536] = data[:,1535]
                data[:,1664] = data[:,1663]
                data[:,1792] = data[:,1791]
                data[:,1920] = data[:,1919]
            # add timestamp to each frame
            timestamp = reader.meta.get_timestamp_by_frame_number(i, schedule)
            # fig.suptitle(r'$\mathit{T\enspace {} s}$'.format(latex_float(timestamp)), x=.85, y=.95)
            fig.suptitle('$\mathbf{t\enspace =\enspace ' + latex_float(timestamp) + 's}$', x=.85, y=.95)
            
            plot.add_image(plot_name, data, cmap=cmap, clim=clim)

            if view: plot.redraw()
            if not dry_run: 
                writer.grab_frame()

            print('DONE')